{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "7f0a5264-b003-4101-862f-45653f2aed1b",
   "metadata": {},
   "source": [
    "# How to write multi-batch `BatchRequest` - Configured `Sql` Example\n",
    "* A `BatchRequest` facilitates the return of a `batch` of data from a configured `Datasource`. To find more about `Batches`, please refer to the [related documentation](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_get_a_batch_of_data_from_a_configured_datasource#1-construct-a-batchrequest). \n",
    "* A `BatchRequest` can return 0 or more Batches of data depending on the underlying data, and how it is configured. This guide will help you configure `BatchRequests` to return multiple batches, which can be used by\n",
    "   1. Self-Initializing Expectations to estimate parameters\n",
    "   2. DataAssistants to profile your data and create and Expectation suite with self-intialized parameters.\n",
    "   \n",
    "* Note : Multi-batch BatchRequests are not supported in `RuntimeDataConnector`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 37,
   "id": "ee54886b-4f88-46d9-9afe-dfd8bb061e19",
   "metadata": {},
   "outputs": [],
   "source": [
    "import great_expectations as gx\n",
    "from great_expectations.core.yaml_handler import YAMLHandler\n",
    "from great_expectations.core.batch import BatchRequest\n",
    "import sqlite3\n",
    "import pprint\n",
    "from ruamel import yaml\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "bfa243d2-6905-403a-b47a-d89ba834b951",
   "metadata": {},
   "source": [
    "* Load `DataContext`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "id": "45b1854b-2a75-422e-83bb-5509d868e0c5",
   "metadata": {},
   "outputs": [],
   "source": [
    "data_context: gx.DataContext = gx.get_context()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c4462320-d76e-492c-96fb-f0ff8f788851",
   "metadata": {},
   "source": [
    "## Sql Example"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "a6e04726",
   "metadata": {},
   "source": [
    "### Example Database\n",
    "\n",
    "Imagine we have a database of 1 table, with `yellow_tripdata_sample_2020`, corresponding to all 12 months' `taxi_trip` data for 2020.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "cd29b60b-7e16-4978-acee-0dab368cde3c",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "['yellow_tripdata_sample_2019', 'yellow_tripdata_sample_2020']\n"
     ]
    }
   ],
   "source": [
    "# connect to sqlite DB, and print the existing tables\n",
    "CONNECTION_STRING = \"postgresql+psycopg2://postgres:@localhost/test_ci\"\n",
    "from sqlalchemy import create_engine\n",
    "from sqlalchemy import inspect\n",
    "engine = create_engine(CONNECTION_STRING)\n",
    "insp = inspect(engine)\n",
    "print(insp.get_table_names())"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "49515697-83a2-432d-8b74-33e2f01db72c",
   "metadata": {},
   "source": [
    "## Example Configuration"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "df19a29c",
   "metadata": {},
   "source": [
    "In our example, we add a `Datasource` named `taxi_multi_batch_sql_datasource` with 1 table. We also have a `ConfiguredAssetSqlDataConnector` named `configured_data_connector_multi_batch_asset`.\n",
    "\n",
    "The DataConnector contains 2 `assets`, both associated with the `table_name` named`yellow_tripdata_sample_2020`.\n",
    "\n",
    "The asset `yellow_tripdata_sample_2020_full` contains no other parameter other than the `table_name` and optional `schema_name`, which mean the whole table will be loaded as one Batch in the asset. \n",
    "\n",
    "The asset `yellow_tripdata_sample_2020_by_year_and_month` contains `table_name` and `schema_name`, as well as a splitter configuration. The splitter we use is `split_on_year_and_month`, which creates Batches according to the `pickup_datetime` column which is of type timestamp in the database schema.\n",
    "\n",
    "**Note**: This example only uses `splitters` but sampling can also be used. For more information, please refer to the document [How to configure a DataConnector for splitting and sampling tables in SQL](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/advanced/how_to_configure_a_dataconnector_for_splitting_and_sampling_tables_in_sql)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 34,
   "id": "84150f65-bbd6-4b45-95ab-9590a29f116a",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Attempting to instantiate class from config...\n",
      "\tInstantiating as a Datasource, since class_name is Datasource\n",
      "\tSuccessfully instantiated Datasource\n",
      "\n",
      "\n",
      "ExecutionEngine class name: SqlAlchemyExecutionEngine\n",
      "Data Connectors:\n",
      "\tconfigured_data_connector_multi_batch_asset : ConfiguredAssetSqlDataConnector\n",
      "\n",
      "\tAvailable data_asset_names (2 of 2):\n",
      "\t\tyellow_tripdata_sample_2020_by_year_and_month (3 of 5): [{'pickup_datetime': {'year': 2020, 'month': 5}}, {'pickup_datetime': {'year': 2020, 'month': 4}}, {'pickup_datetime': {'year': 2020, 'month': 3}}]\n",
      "\t\tyellow_tripdata_sample_2020_full (1 of 1): [{}]\n",
      "\n",
      "\tUnmatched data_references (0 of 0):[]\n",
      "\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<great_expectations.datasource.new_datasource.Datasource at 0x7ff1c46ec160>"
      ]
     },
     "execution_count": 34,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "datasource_config = {\n",
    "    \"name\": \"taxi_multi_batch_sql_datasource\",\n",
    "    \"class_name\": \"Datasource\",\n",
    "    \"module_name\": \"great_expectations.datasource\",\n",
    "    \"execution_engine\": {\n",
    "        \"module_name\": \"great_expectations.execution_engine\",\n",
    "        \"class_name\": \"SqlAlchemyExecutionEngine\",\n",
    "        \"connection_string\": CONNECTION_STRING,\n",
    "    },\n",
    "    \"data_connectors\": {\n",
    "        \"configured_data_connector_multi_batch_asset\": {\n",
    "            \"class_name\": \"ConfiguredAssetSqlDataConnector\",\n",
    "            \"assets\":{\n",
    "                \"yellow_tripdata_sample_2020_full\":\n",
    "                {\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                },\n",
    "    \n",
    "                \"yellow_tripdata_sample_2020_by_year_and_month\":{\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                    \"splitter_method\": \"split_on_year_and_month\",\n",
    "                    \"splitter_kwargs\": {\n",
    "                        \"column_name\": \"pickup_datetime\",\n",
    "                        },\n",
    "                    },\n",
    "                },\n",
    "            },\n",
    "            \n",
    "        },\n",
    "}\n",
    "\n",
    "data_context.test_yaml_config(yaml.dump(datasource_config))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "635755a4-36b1-442f-968a-2fbdb9b146d8",
   "metadata": {},
   "source": [
    "We see we have successfully configured this because the output shows a 2 data assets\n",
    "- `yellow_tripdata_sample_2020_full` associated with 1 batch. \n",
    "- `yellow_tripdata_sample_2020_by_year_and_month` with 12 batches, each associated with a different month in our `pickup_datetime` column. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "7636ffff-0ddc-48fd-a4cf-f8e139a5e36e",
   "metadata": {},
   "outputs": [],
   "source": [
    "# add_datasource only if it doesn't already exist in our configuration\n",
    "try:\n",
    "    data_context.get_datasource(datasource_config[\"name\"])\n",
    "except ValueError:\n",
    "    data_context.add_datasource(**datasource_config)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "438146be",
   "metadata": {},
   "source": [
    "## BatchRequest"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "117cdb0d",
   "metadata": {},
   "source": [
    "Depending on how we configured our assets, when you send a `BatchRequest`, you will retrieve a different number of `Batches`"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "87bc9c59",
   "metadata": {},
   "source": [
    "Single Batch returned by `yellow_tripdata_sample_2020_full`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "0453cd3c",
   "metadata": {},
   "outputs": [],
   "source": [
    "single_batch_batch_request: BatchRequest = BatchRequest(\n",
    "    datasource_name=\"taxi_multi_batch_sql_datasource\",\n",
    "    data_connector_name=\"configured_data_connector_multi_batch_asset\",\n",
    "    data_asset_name=\"yellow_tripdata_sample_2020_full\",\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "35df7084",
   "metadata": {},
   "outputs": [],
   "source": [
    "batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "d8e74a05-3fd1-4e47-9105-b721dbcf3516",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<great_expectations.core.batch.Batch at 0x7ff1c43336a0>]"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "batch_list"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "be7f4fa5",
   "metadata": {},
   "source": [
    "Multi Batch returned by `yellow_tripdata_sample_2020_by_year_and_month`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "id": "c32dbac9-af5d-4677-98f9-f098ef091b6f",
   "metadata": {},
   "outputs": [],
   "source": [
    "multi_batch_batch_request: BatchRequest = BatchRequest(\n",
    "    datasource_name=\"taxi_multi_batch_sql_datasource\",\n",
    "    data_connector_name=\"configured_data_connector_multi_batch_asset\",\n",
    "    data_asset_name=\"yellow_tripdata_sample_2020_by_year_and_month\",\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "id": "3a284bfd-00aa-4068-bc09-71c6dea627e0",
   "metadata": {},
   "outputs": [],
   "source": [
    "multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "id": "7bf3e1ff",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<great_expectations.core.batch.Batch at 0x7ff1c46f4730>,\n",
       " <great_expectations.core.batch.Batch at 0x7ff1c429b940>,\n",
       " <great_expectations.core.batch.Batch at 0x7ff1c45794f0>,\n",
       " <great_expectations.core.batch.Batch at 0x7ff1c469bac0>,\n",
       " <great_expectations.core.batch.Batch at 0x7ff1c42aa370>]"
      ]
     },
     "execution_count": 33,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "multi_batch_batch_list # 12 batches"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "790746ee",
   "metadata": {},
   "source": [
    "You can also get a single Batch from a multi-batch DataConnector by passing in `data_connector_query`. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "16612bb4",
   "metadata": {},
   "outputs": [],
   "source": [
    "single_batch_batch_request_from_multi: BatchRequest = BatchRequest(\n",
    "    datasource_name=\"taxi_multi_batch_sql_datasource\",\n",
    "    data_connector_name=\"configured_data_connector_multi_batch_asset\",\n",
    "    data_asset_name=\"yellow_tripdata_sample_2020_by_year_and_month\",\n",
    "    data_connector_query={ \n",
    "        \"batch_filter_parameters\": {\"pickup_datetime\": {\"year\": 2020, \"month\": 1}}\n",
    "    }\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "6ef1b6a9",
   "metadata": {},
   "outputs": [],
   "source": [
    "batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request_from_multi)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "80a69e96",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[<great_expectations.core.batch.Batch at 0x7ff1dbe703d0>]"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "batch_list # has a length of 1, as expected"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "26cd5a37",
   "metadata": {},
   "source": [
    "Let's review our batch:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "229815cd",
   "metadata": {},
   "outputs": [],
   "source": [
    "batch = batch_list[0]  # our single filtered batch with 'batch_identifiers': {'pickup_datetime': '2020-01'}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "id": "f2be5bdb",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'data': '<great_expectations.execution_engine.sqlalchemy_batch_data.SqlAlchemyBatchData object at 0x7ff1c42fb340>',\n",
       " 'batch_request': {'datasource_name': 'taxi_multi_batch_sql_datasource',\n",
       "  'data_connector_name': 'configured_data_connector_multi_batch_asset',\n",
       "  'data_asset_name': 'yellow_tripdata_sample_2020_by_year_and_month',\n",
       "  'limit': None,\n",
       "  'batch_spec_passthrough': None,\n",
       "  'data_connector_query': {'batch_filter_parameters': {'pickup_datetime': {'year': 2020,\n",
       "     'month': 1}}}},\n",
       " 'batch_definition': {'datasource_name': 'taxi_multi_batch_sql_datasource',\n",
       "  'data_connector_name': 'configured_data_connector_multi_batch_asset',\n",
       "  'data_asset_name': 'yellow_tripdata_sample_2020_by_year_and_month',\n",
       "  'batch_identifiers': {'pickup_datetime': {'year': 2020, 'month': 1}}},\n",
       " 'batch_spec': {'data_asset_name': 'yellow_tripdata_sample_2020_by_year_and_month',\n",
       "  'table_name': 'yellow_tripdata_sample_2020',\n",
       "  'batch_identifiers': {'pickup_datetime': {'year': 2020, 'month': 1}},\n",
       "  'class_name': 'Asset',\n",
       "  'schema_name': 'public',\n",
       "  'splitter_method': 'split_on_year_and_month',\n",
       "  'module_name': 'great_expectations.datasource.data_connector.asset',\n",
       "  'splitter_kwargs': {'column_name': 'pickup_datetime'},\n",
       "  'type': None},\n",
       " 'batch_markers': {'ge_load_time': '20220913T024332.529136Z'}}"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "batch.to_dict()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "907a6ef5",
   "metadata": {},
   "source": [
    "# Using auto-initializing `Expectations` to generate parameters"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6efe9c76",
   "metadata": {},
   "source": [
    "We will generate a `Validator` using our `multi_batch_batch_list`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "847ce4a3",
   "metadata": {},
   "outputs": [],
   "source": [
    "multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "a1eca55b",
   "metadata": {},
   "outputs": [],
   "source": [
    "example_suite = data_context.create_expectation_suite(expectation_suite_name=\"example_sql_suite\", overwrite_existing=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "852deba1",
   "metadata": {},
   "outputs": [],
   "source": [
    "validator = data_context.get_validator_using_batch_list(batch_list=multi_batch_batch_list, expectation_suite=example_suite)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ee01a0e9",
   "metadata": {},
   "source": [
    "When you run methods on the validator, it will typically run on the most recent batch (index `-1`), even if the Validator has access to a longer Batch list. For example, notice that rows below are all associated with `pickup_datetime` being `9` (September, 2020). This is because the datetime values are stored lexicographically, meaning `1` and `11`, `12` values will appear **before** `2` and `3`."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d8d7d827-ed51-4f83-ac41-33ae58416ef1",
   "metadata": {},
   "source": [
    "For simplicity, let's get a `validator` with the December `Batch`, which is in index `\"3\"` (after `1`, `10`, `11`). Notice that we are also casting the value as a `list` using the square brackets. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "11b673bc-8583-4fc5-9aa0-db6ca62e240c",
   "metadata": {},
   "outputs": [],
   "source": [
    "validator = data_context.get_validator_using_batch_list(batch_list=[multi_batch_batch_list[3]], expectation_suite=example_suite)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "ffe9cabd-b2bd-46de-9317-ba4e809342fa",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "7f1f7a3134bc4325b15d750c12b018bf",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>vendor_id</th>\n",
       "      <th>pickup_datetime</th>\n",
       "      <th>dropoff_datetime</th>\n",
       "      <th>passenger_count</th>\n",
       "      <th>trip_distance</th>\n",
       "      <th>rate_code_id</th>\n",
       "      <th>store_and_fwd_flag</th>\n",
       "      <th>pickup_location_id</th>\n",
       "      <th>dropoff_location_id</th>\n",
       "      <th>payment_type</th>\n",
       "      <th>fare_amount</th>\n",
       "      <th>extra</th>\n",
       "      <th>mta_tax</th>\n",
       "      <th>tip_amount</th>\n",
       "      <th>tolls_amount</th>\n",
       "      <th>improvement_surcharge</th>\n",
       "      <th>total_amount</th>\n",
       "      <th>congestion_surcharge</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>2.0</td>\n",
       "      <td>2020-04-03 16:23:46</td>\n",
       "      <td>2020-04-03 16:34:42</td>\n",
       "      <td>5.0</td>\n",
       "      <td>2.26</td>\n",
       "      <td>1.0</td>\n",
       "      <td>N</td>\n",
       "      <td>263</td>\n",
       "      <td>41</td>\n",
       "      <td>1.0</td>\n",
       "      <td>10.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.5</td>\n",
       "      <td>2.86</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.3</td>\n",
       "      <td>17.16</td>\n",
       "      <td>2.5</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>1.0</td>\n",
       "      <td>2020-04-29 09:45:35</td>\n",
       "      <td>2020-04-29 09:48:36</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.40</td>\n",
       "      <td>1.0</td>\n",
       "      <td>N</td>\n",
       "      <td>43</td>\n",
       "      <td>151</td>\n",
       "      <td>1.0</td>\n",
       "      <td>4.0</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.5</td>\n",
       "      <td>0.00</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.3</td>\n",
       "      <td>4.80</td>\n",
       "      <td>0.0</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>1.0</td>\n",
       "      <td>2020-04-27 19:21:48</td>\n",
       "      <td>2020-04-27 19:29:07</td>\n",
       "      <td>1.0</td>\n",
       "      <td>2.80</td>\n",
       "      <td>1.0</td>\n",
       "      <td>N</td>\n",
       "      <td>140</td>\n",
       "      <td>107</td>\n",
       "      <td>1.0</td>\n",
       "      <td>10.0</td>\n",
       "      <td>3.5</td>\n",
       "      <td>0.5</td>\n",
       "      <td>3.55</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.3</td>\n",
       "      <td>17.85</td>\n",
       "      <td>2.5</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>2.0</td>\n",
       "      <td>2020-04-23 18:01:29</td>\n",
       "      <td>2020-04-23 18:06:14</td>\n",
       "      <td>1.0</td>\n",
       "      <td>1.03</td>\n",
       "      <td>1.0</td>\n",
       "      <td>N</td>\n",
       "      <td>142</td>\n",
       "      <td>238</td>\n",
       "      <td>1.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.5</td>\n",
       "      <td>2.58</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.3</td>\n",
       "      <td>12.88</td>\n",
       "      <td>2.5</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>1.0</td>\n",
       "      <td>2020-04-29 09:02:24</td>\n",
       "      <td>2020-04-29 09:08:48</td>\n",
       "      <td>1.0</td>\n",
       "      <td>0.80</td>\n",
       "      <td>1.0</td>\n",
       "      <td>N</td>\n",
       "      <td>161</td>\n",
       "      <td>100</td>\n",
       "      <td>1.0</td>\n",
       "      <td>6.0</td>\n",
       "      <td>2.5</td>\n",
       "      <td>0.5</td>\n",
       "      <td>2.30</td>\n",
       "      <td>0.0</td>\n",
       "      <td>0.3</td>\n",
       "      <td>11.60</td>\n",
       "      <td>2.5</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "   vendor_id     pickup_datetime    dropoff_datetime  passenger_count  \\\n",
       "0        2.0 2020-04-03 16:23:46 2020-04-03 16:34:42              5.0   \n",
       "1        1.0 2020-04-29 09:45:35 2020-04-29 09:48:36              0.0   \n",
       "2        1.0 2020-04-27 19:21:48 2020-04-27 19:29:07              1.0   \n",
       "3        2.0 2020-04-23 18:01:29 2020-04-23 18:06:14              1.0   \n",
       "4        1.0 2020-04-29 09:02:24 2020-04-29 09:08:48              1.0   \n",
       "\n",
       "   trip_distance  rate_code_id store_and_fwd_flag  pickup_location_id  \\\n",
       "0           2.26           1.0                  N                 263   \n",
       "1           0.40           1.0                  N                  43   \n",
       "2           2.80           1.0                  N                 140   \n",
       "3           1.03           1.0                  N                 142   \n",
       "4           0.80           1.0                  N                 161   \n",
       "\n",
       "   dropoff_location_id  payment_type  fare_amount  extra  mta_tax  tip_amount  \\\n",
       "0                   41           1.0         10.0    1.0      0.5        2.86   \n",
       "1                  151           1.0          4.0    0.0      0.5        0.00   \n",
       "2                  107           1.0         10.0    3.5      0.5        3.55   \n",
       "3                  238           1.0          6.0    1.0      0.5        2.58   \n",
       "4                  100           1.0          6.0    2.5      0.5        2.30   \n",
       "\n",
       "   tolls_amount  improvement_surcharge  total_amount  congestion_surcharge  \n",
       "0           0.0                    0.3         17.16                   2.5  \n",
       "1           0.0                    0.3          4.80                   0.0  \n",
       "2           0.0                    0.3         17.85                   2.5  \n",
       "3           0.0                    0.3         12.88                   2.5  \n",
       "4           0.0                    0.3         11.60                   2.5  "
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "validator.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "8be66602",
   "metadata": {},
   "source": [
    "### Typical Workflow\n",
    "A `batch_list` becomes really useful when you are calculating parameters for auto-initializing Expectations, as they use a `RuleBasedProfiler` under-the-hood to calculate parameters."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "fdef9ad7",
   "metadata": {},
   "source": [
    "Let's say we don't know the `min_value` and `max_value` for `expect_column_median_to_be_between()` so we \"guess\" at the `min_value` and `max_value`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "d7642647",
   "metadata": {},
   "outputs": [],
   "source": [
    "validator = data_context.get_validator_using_batch_list(batch_list=multi_batch_batch_list, expectation_suite=example_suite)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "b524a2df",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "bdc46d7f52834ae9a8dcea9d19d51689",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "{\n",
       "  \"success\": false,\n",
       "  \"result\": {\n",
       "    \"observed_value\": 1.99\n",
       "  },\n",
       "  \"meta\": {},\n",
       "  \"exception_info\": {\n",
       "    \"raised_exception\": false,\n",
       "    \"exception_traceback\": null,\n",
       "    \"exception_message\": null\n",
       "  }\n",
       "}"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "validator.expect_column_median_to_be_between(column=\"trip_distance\", min_value=0, max_value=1)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f18d096f",
   "metadata": {},
   "source": [
    "The observed value of `trip_distance` for our `yellow_tripdata_sample_2020` going to be `1.75`, which means the Expectation fails. We guessed wrong - but we can do better!"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9c5016d8",
   "metadata": {},
   "source": [
    "Now we run the same expectation again, but this time with `auto=True`. This means the median values are going to calculated across the `batch_list` associated with the `Validator` (ie 12 Batches for `yellow_tripdata_sample_2020`), which gives the min value of `1.6` and the max value of `1.99`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "cdd821c3",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n"
     ]
    },
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "7aa4929f2b514494af1dc0ffdf7a9f5f",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Generating Expectations:   0%|          | 0/1 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "44d2b5cb838845a4a677bb74ad7b45e5",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    },
    {
     "data": {
      "text/plain": [
       "{\n",
       "  \"success\": true,\n",
       "  \"expectation_config\": {\n",
       "    \"expectation_type\": \"expect_column_median_to_be_between\",\n",
       "    \"kwargs\": {\n",
       "      \"column\": \"trip_distance\",\n",
       "      \"min_value\": 1.6,\n",
       "      \"max_value\": 1.99,\n",
       "      \"strict_min\": false,\n",
       "      \"strict_max\": false\n",
       "    },\n",
       "    \"meta\": {\n",
       "      \"auto_generated_at\": \"20220913T024338.419229Z\",\n",
       "      \"great_expectations_version\": \"0.15.22+29.g8fc1586df\"\n",
       "    }\n",
       "  },\n",
       "  \"result\": {\n",
       "    \"observed_value\": 1.99\n",
       "  },\n",
       "  \"meta\": {},\n",
       "  \"exception_info\": {\n",
       "    \"raised_exception\": false,\n",
       "    \"exception_traceback\": null,\n",
       "    \"exception_message\": null\n",
       "  }\n",
       "}"
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "validator.expect_column_median_to_be_between(column=\"trip_distance\", auto=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c6a6c277",
   "metadata": {},
   "source": [
    "The auto=True will also automatically run the Expectation against the most recent Batch (which has an observed value of `1.75`) and the Expectation will pass."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "90b8b938",
   "metadata": {},
   "source": [
    "You can now save the `ExpectationSuite`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "id": "eba880ed",
   "metadata": {},
   "outputs": [],
   "source": [
    "validator.save_expectation_suite()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1b7ec631",
   "metadata": {},
   "source": [
    "### Running the `ExpectationSuite` against single `Batch`"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "477381f5",
   "metadata": {},
   "source": [
    "Now the ExpectationSuite we built using all batches can be used to validate single batches using a Checkpoint. For example, we can run this checkpoint on new data when it comes in next month. In our example, let's validatidate a different batch from February 2020, using the ExpectationSuite we built from `yellow_tripdata_sample_2020`.\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "id": "747dea95",
   "metadata": {},
   "outputs": [],
   "source": [
    "single_batch_batch_request_from_multi: BatchRequest = BatchRequest(\n",
    "    datasource_name=\"taxi_multi_batch_sql_datasource\",\n",
    "    data_connector_name=\"configured_data_connector_multi_batch_asset\",\n",
    "    data_asset_name=\"yellow_tripdata_sample_2020_by_year_and_month\",\n",
    "    data_connector_query={ \n",
    "        \"batch_filter_parameters\": {\"pickup_datetime\": {\"year\": 2020, \"month\": 2}}\n",
    "    }\n",
    "\n",
    "\n",
    ")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "id": "cb93d87f",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{\n",
       "  \"action_list\": [\n",
       "    {\n",
       "      \"name\": \"store_validation_result\",\n",
       "      \"action\": {\n",
       "        \"class_name\": \"StoreValidationResultAction\"\n",
       "      }\n",
       "    },\n",
       "    {\n",
       "      \"name\": \"store_evaluation_params\",\n",
       "      \"action\": {\n",
       "        \"class_name\": \"StoreEvaluationParametersAction\"\n",
       "      }\n",
       "    },\n",
       "    {\n",
       "      \"name\": \"update_data_docs\",\n",
       "      \"action\": {\n",
       "        \"class_name\": \"UpdateDataDocsAction\",\n",
       "        \"site_names\": []\n",
       "      }\n",
       "    }\n",
       "  ],\n",
       "  \"batch_request\": {},\n",
       "  \"class_name\": \"Checkpoint\",\n",
       "  \"config_version\": 1.0,\n",
       "  \"evaluation_parameters\": {},\n",
       "  \"module_name\": \"great_expectations.checkpoint\",\n",
       "  \"name\": \"my_checkpoint\",\n",
       "  \"profilers\": [],\n",
       "  \"runtime_configuration\": {},\n",
       "  \"validations\": [\n",
       "    {\n",
       "      \"batch_request\": {\n",
       "        \"datasource_name\": \"taxi_multi_batch_sql_datasource\",\n",
       "        \"data_connector_name\": \"configured_data_connector_multi_batch_asset\",\n",
       "        \"data_asset_name\": \"yellow_tripdata_sample_2020_by_year_and_month\",\n",
       "        \"data_connector_query\": {\n",
       "          \"batch_filter_parameters\": {\n",
       "            \"pickup_datetime\": {\n",
       "              \"year\": 2020,\n",
       "              \"month\": 2\n",
       "            }\n",
       "          }\n",
       "        }\n",
       "      },\n",
       "      \"expectation_suite_name\": \"example_sql_suite\"\n",
       "    }\n",
       "  ]\n",
       "}"
      ]
     },
     "execution_count": 27,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "checkpoint_config = {\n",
    "    \"name\": \"my_checkpoint\",\n",
    "    \"config_version\": 1,\n",
    "    \"class_name\": \"SimpleCheckpoint\",\n",
    "    \"validations\": [\n",
    "        {\n",
    "            \"batch_request\": single_batch_batch_request_from_multi,\n",
    "            \"expectation_suite_name\": \"example_sql_suite\",            \n",
    "        }\n",
    "    ],\n",
    "}\n",
    "data_context.add_checkpoint(**checkpoint_config)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "id": "3269dfba",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "dda39f32a2424457bb73f1197bd904f3",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "results = data_context.run_checkpoint(\n",
    "    checkpoint_name=\"my_checkpoint\"\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "id": "c6234082",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "True"
      ]
     },
     "execution_count": 29,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "results.success"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6737ccdd",
   "metadata": {},
   "source": [
    "# Appendix\n"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6914725d-85cc-4c9d-a93d-2d1c329eac74",
   "metadata": {},
   "source": [
    "## Other Parameters for `ConfiguredAssetSqlDataConnector`"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "f8e62691",
   "metadata": {},
   "source": [
    "The signature of the `ConfiguredAssetSqlDataConnector` also contains the following parameters: \n",
    "\n",
    "The following required parameters:\n",
    "* `name`: The name of this DataConnector.\n",
    "* `datasource_name`: The name of the Datasource that contains it.\n",
    "* `execution_engine`: the type of ExecutionEngine to use.\n",
    "* `assets`: The dictionary containing the asset configurations.\n",
    "\n",
    "The `assets` dictionary can contain the following keys and values:\n",
    "* `table_name`: string that defines the `table_name` associated with the asset. If table_name is omitted, then the `table_name` defaults to the asset name.\n",
    "* `schema_name`: optional string that defines the `schema` for the asset.\n",
    "* `include_schema_name`: A `bool` that determines, \"Should the `data_asset_name` include the `schema` as a prefix?\"\n",
    "* `splitter_method`: string that names method to split the target table into multiple `Batches`.\n",
    "* `splitter_kwargs`: a dict containing arguments to pass to `splitter_method`.\n",
    "* `sampling_method`: string that names method to downsample within a target `Batch`.\n",
    "* `sampling_kwargs` : dictionary with keyword arguments to pass to `sampling_method`.\n",
    "* `batch_spec_passthrough`: dictionary with keys that will be added directly to `batch_spec`.\n",
    "\n",
    "\n",
    "For more information on `splitters` and `samplers` please consider the following documentation: [How to configure a DataConnector for splitting and sampling tables in SQL](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/advanced/how_to_configure_a_dataconnector_for_splitting_and_sampling_tables_in_sql)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "10a27510",
   "metadata": {},
   "source": [
    "## Configuring Splitters at `DataConnector` and `Asset`-Level"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7fb9c585",
   "metadata": {},
   "source": [
    "For `ConfiguredAssetSqlDataConnectors`, the `splitter_method` and `splitter_kwargs` can be configured at the `DataConnector`-level or `Asset`-level. "
   ]
  },
  {
   "cell_type": "markdown",
   "id": "263ed79f",
   "metadata": {},
   "source": [
    "#### Configuration at `DataConnector`-level"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "baad2c0d",
   "metadata": {},
   "source": [
    "\n",
    "Here is a configuration with the splitter method `split_on_year_and_month` configured at the `DataConnector`-level for a `DataConnector` with 2 `Assets`, `yellow_tripdata_sample_2020_by_year_and_month` and `yellow_tripdata_sample_2020`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "id": "85b9598e",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Attempting to instantiate class from config...\n",
      "\tInstantiating as a Datasource, since class_name is Datasource\n",
      "\tSuccessfully instantiated Datasource\n",
      "\n",
      "\n",
      "ExecutionEngine class name: SqlAlchemyExecutionEngine\n",
      "Data Connectors:\n",
      "\tconfigured_data_connector_multi_batch_asset : ConfiguredAssetSqlDataConnector\n",
      "\n",
      "\tAvailable data_asset_names (2 of 2):\n",
      "\t\tyellow_tripdata_sample_2020 (3 of 5): [{'pickup_datetime': {'year': 2020, 'month': 5}}, {'pickup_datetime': {'year': 2020, 'month': 4}}, {'pickup_datetime': {'year': 2020, 'month': 3}}]\n",
      "\t\tyellow_tripdata_sample_2020_by_year_and_month (3 of 5): [{'pickup_datetime': {'year': 2020, 'month': 5}}, {'pickup_datetime': {'year': 2020, 'month': 4}}, {'pickup_datetime': {'year': 2020, 'month': 3}}]\n",
      "\n",
      "\tUnmatched data_references (0 of 0):[]\n",
      "\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "<great_expectations.datasource.new_datasource.Datasource at 0x7ff1c4564070>"
      ]
     },
     "execution_count": 30,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "datasource_config = {\n",
    "    \"name\": \"taxi_multi_batch_sql_datasource\",\n",
    "    \"class_name\": \"Datasource\",\n",
    "    \"module_name\": \"great_expectations.datasource\",\n",
    "    \"execution_engine\": {\n",
    "        \"module_name\": \"great_expectations.execution_engine\",\n",
    "        \"class_name\": \"SqlAlchemyExecutionEngine\",\n",
    "        \"connection_string\": CONNECTION_STRING,\n",
    "    },\n",
    "    \"data_connectors\": {\n",
    "        \"configured_data_connector_multi_batch_asset\": {\n",
    "            \"class_name\": \"ConfiguredAssetSqlDataConnector\",\n",
    "            \"splitter_method\": \"split_on_year_and_month\",\n",
    "            \"splitter_kwargs\": {\n",
    "                \"column_name\": \"pickup_datetime\",\n",
    "            },\n",
    "            \"assets\":{\n",
    "    \n",
    "                \"yellow_tripdata_sample_2020_by_year_and_month\":{\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                },\n",
    "                \"yellow_tripdata_sample_2020\":{\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                },\n",
    "            },\n",
    "            \n",
    "        },\n",
    "    }\n",
    "}\n",
    "\n",
    "data_context.test_yaml_config(yaml.dump(datasource_config))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "2f36497e",
   "metadata": {},
   "source": [
    "As you can see, both `Assets`, `yellow_tripdata_sample_2020_by_year_and_month` **and** `yellow_tripdata_sample_2020` have the splitter method applied to it, meaning they both have 12 Batches as a result of splitting by `year` and `month`."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "162c74bc",
   "metadata": {},
   "source": [
    "#### Configuration at `DataConnector`-level **and** `Asset`-level"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "7d2134d3",
   "metadata": {},
   "source": [
    "Next we have a similar example, but with a second `splitter_method` also configured at the `Asset`-level. This time we will configure a second `splitter_method`, `split_on_year_and_month_and_day`, for the Asset `yellow_tripdata_sample_2020_by_year_and_month_and_day`. In this case, the `Asset`-level configuration will **override** the configuration at the `DataConnector`-level and produce 366 Batches as a result of splitting by `year`, `month` and `day`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "bd803494",
   "metadata": {},
   "outputs": [],
   "source": [
    "datasource_config = {\n",
    "    \"name\": \"taxi_multi_batch_sql_datasource\",\n",
    "    \"class_name\": \"Datasource\",\n",
    "    \"module_name\": \"great_expectations.datasource\",\n",
    "    \"execution_engine\": {\n",
    "        \"module_name\": \"great_expectations.execution_engine\",\n",
    "        \"class_name\": \"SqlAlchemyExecutionEngine\",\n",
    "        \"connection_string\": CONNECTION_STRING,\n",
    "    },\n",
    "    \"data_connectors\": {\n",
    "        \"configured_data_connector_multi_batch_asset\": {\n",
    "            \"class_name\": \"ConfiguredAssetSqlDataConnector\",\n",
    "            \"splitter_method\": \"split_on_year_and_month\",\n",
    "            \"splitter_kwargs\": {\n",
    "                \"column_name\": \"pickup_datetime\",\n",
    "            },\n",
    "            \"assets\":{\n",
    "    \n",
    "                \"yellow_tripdata_sample_2020_by_year_and_month\":{\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                },\n",
    "                \"yellow_tripdata_sample_2020_by_year_and_month_and_day\":{\n",
    "                    \"table_name\": \"yellow_tripdata_sample_2020\",\n",
    "                    \"schema_name\": \"public\",\n",
    "                    \"splitter_method\": \"split_on_year_and_month_and_day\",\n",
    "                    \"splitter_kwargs\": {\n",
    "                        \"column_name\": \"pickup_datetime\",\n",
    "                    },\n",
    "                },\n",
    "            },\n",
    "            \n",
    "        },\n",
    "    }\n",
    "}\n",
    "\n",
    "data_context.test_yaml_config(yaml.dump(datasource_config))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "80ff49fe",
   "metadata": {},
   "source": [
    "As you can see, `yellow_tripdata_sample_2020_by_year_and_month` and `yellow_tripdata_sample_2020_by_year_and_month_and_day` each have a different number of Batches resulting from their different `splitter` configurations. \n",
    "\n",
    "* `yellow_tripdata_sample_2020_by_year_and_month` has 12 Batches. \n",
    "* `yellow_tripdata_sample_2020_by_year_and_month_and_day` has 366 Batches."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "83ecebdc",
   "metadata": {},
   "source": [
    "# Loading Data into Postgresql Database"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "9f4a183e-f0df-40f4-b64f-439666e32ab4",
   "metadata": {},
   "source": [
    "* The following code can be used to build the postgres database used in this notebook. It is included (and commented out) for reference.\n",
    "* In order to load the data into a local `postgresql` database, please feel free to use the `docker-compose.yml` file available at `great_expectations/assets/docker/postgresql/`. \n",
    "\n",
    "### To spin up the `postgresql` database\n",
    "* Have [Docker Desktop](https://www.docker.com/products/docker-desktop/) running locally.\n",
    "* Navigate to `great_expectations/assets/docker/postgresql/`\n",
    "* Type `docker-compose up`\n",
    "* Then uncomment and run the following snippet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "id": "ad91c234-06a2-4e98-9d90-c999c2aad07f",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-01.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-02.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-03.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-04.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-05.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-06.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-07.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-08.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-09.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-10.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-11.csv']\n",
      "Adding to existing table yellow_tripdata_sample_2020 and adding data from ['../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-12.csv']\n"
     ]
    }
   ],
   "source": [
    "from tests.test_utils import load_data_into_test_database\n",
    "from typing import List\n",
    "import sqlalchemy as sa\n",
    "import pandas as pd\n",
    "CONNECTION_STRING = \"postgresql+psycopg2://postgres:@localhost/test_ci\"\n",
    "\n",
    "data_paths: List[str] = [\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-01.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-02.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-03.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-04.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-05.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-06.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-07.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-08.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-09.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-10.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-11.csv\",\n",
    "     \"../../../test_sets/taxi_yellow_tripdata_samples/yellow_tripdata_sample_2020-12.csv\",\n",
    "]\n",
    "\n",
    "    \n",
    "engine = sa.create_engine(CONNECTION_STRING)\n",
    "connection = engine.connect()\n",
    "table_name = \"yellow_tripdata_sample_2020\"\n",
    "res = connection.execute(f\"DROP TABLE IF EXISTS {table_name}\")\n",
    "\n",
    "for data_path in data_paths:\n",
    "    # This utility is not for general use. It is only to support testing.\n",
    "    load_data_into_test_database(\n",
    "        table_name=\"yellow_tripdata_sample_2020\",\n",
    "        csv_path=data_path,\n",
    "        connection_string=CONNECTION_STRING,\n",
    "        load_full_dataset=True,\n",
    "        drop_existing_table=False,\n",
    "        convert_colnames_to_datetime=[\"pickup_datetime\", \"dropoff_datetime\"]\n",
    "    )"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6d4815f2",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}